In [131]:
import xml.etree.ElementTree as ET
import pandas as pd
tree = ET.parse('Leer_planned09.xml')
root = tree.getroot()

# Initialize lists to store data
data = []

# Iterate over each 's' element in the XML
for s in root.findall('.//s'):
    s_id = s.get('id')
    s_eva = s.get('eva')
    # Extract 'ar' element data
    ar = s.find('ar')
    if ar is not None:
        ar_pt = ar.get('pt')#planned arrival
        ar_l = ar.get('l')
    else:
        ar_ct = ar_l = None
    # Extract 'dp' element data
    dp = s.find('dp')
    if dp is not None:
        dp_pt = dp.get('pt')
        dp_l = dp.get('l')
    else:
        dp_ct = dp_l = None  
    # Append the extracted data to the list
    data.append([s_id, ar_pt, dp_pt, dp_l])


columns = ['ID', 'arrival', "departure", "train"]
df1 = pd.DataFrame(data, columns=columns)
df1['arrival'] = pd.to_datetime(df1['arrival'], format='%y%m%d%H%M')
df1['departure'] = pd.to_datetime(df1['departure'], format='%y%m%d%H%M')
df1.head()


,ID,arrival,departure,train
0,8863466736973729393-2406190753-8,2024-06-19 09:14:00,2024-06-19 09:22:00,56
1,-1346881581325835580-2406190500-15,2024-06-19 09:14:00,2024-06-19 09:22:00,None
2,-300313760802474975-2406190852-2,2024-06-19 09:08:00,2024-06-19 09:09:00,RE15
3,8982563108078925395-2406190839-6,2024-06-19 09:34:00,2024-06-19 09:41:00,1
4,7350926981163673022-2406190805-15,2024-06-19 09:52:00,2024-06-19 09:53:00,RE15


In [132]:
tree = ET.parse('Leer_changed09.xml')
root = tree.getroot()
# Initialize lists to store data
data = []

# Iterate over each 's' element in the XML
for s in root.findall('.//s'):
    s_id = s.get('id')
    s_eva = s.get('eva')
    # Extract 'ar' element data
    ar = s.find('ar')
    if ar is not None:
        ar_pt = ar.get('ct')#planned arrival
        ar_l = ar.get('l')
    else:
        ar_ct = ar_l = None
    
    # Extract 'dp' element data
    dp = s.find('dp')
    if dp is not None:
        dp_pt = dp.get('ct')
        dp_l = dp.get('l')
    else:
        dp_ct = dp_l = None
    
    # Append the extracted data to the list
    data.append([s_id, s_eva, ar_pt, dp_pt])


columns = ['ID', 'EVANr', 'arrival', "departure",]
df2 = pd.DataFrame(data, columns=columns)
df2['arrival'] = pd.to_datetime(df2['arrival'], format='%y%m%d%H%M')
df2['departure'] = pd.to_datetime(df2['departure'], format='%y%m%d%H%M')
df2.head()

,ID,EVANr,arrival,departure
0,-5246782978511013552-2406190905-15,8000225,2024-06-19 10:54:00,2024-06-19 10:55:00
1,7350926981163673022-2406190805-15,8000225,2024-06-19 10:02:00,2024-06-19 10:03:00
2,6377312335026775072-2406190720-17,8000225,2024-06-19 10:15:00,2024-06-19 10:25:00
3,8982563108078925395-2406190839-6,8000225,2024-06-19 09:35:00,2024-06-19 09:42:00
4,6512907201443103756-2406190926-17,8000225,2024-06-19 15:07:00,2024-06-19 15:09:00


In [133]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ID         73 non-null     object        
 1   EVANr      73 non-null     object        
 2   arrival    67 non-null     datetime64[ns]
 3   departure  67 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.4+ KB


In [134]:
df3 = pd.merge(df1, df2, how='left', on="ID", suffixes=('_plan', '_change'))
df3['depature_delay_m'] = df3['departure_change'] - df3['departure_plan']
df3['arrival_delay_m'] = df3['arrival_change'] - df3['arrival_plan']

In [135]:
df3.head()

,ID,arrival_plan,departure_plan,train,EVANr,arrival_change,departure_change,depature_delay_m,arrival_delay_m
0,8863466736973729393-2406190753-8,2024-06-19 09:14:00,2024-06-19 09:22:00,56,8000225,2024-06-19 09:15:00,2024-06-19 09:22:00,0 days 00:00:00,0 days 00:01:00
1,-1346881581325835580-2406190500-15,2024-06-19 09:14:00,2024-06-19 09:22:00,None,8000225,2024-06-19 09:15:00,2024-06-19 09:24:00,0 days 00:02:00,0 days 00:01:00
2,-300313760802474975-2406190852-2,2024-06-19 09:08:00,2024-06-19 09:09:00,RE15,8000225,2024-06-19 09:08:00,2024-06-19 09:09:00,0 days 00:00:00,0 days 00:00:00
3,8982563108078925395-2406190839-6,2024-06-19 09:34:00,2024-06-19 09:41:00,1,8000225,2024-06-19 09:35:00,2024-06-19 09:42:00,0 days 00:01:00,0 days 00:01:00
4,7350926981163673022-2406190805-15,2024-06-19 09:52:00,2024-06-19 09:53:00,RE15,8000225,2024-06-19 10:02:00,2024-06-19 10:03:00,0 days 00:10:00,0 days 00:10:00


In [136]:
df4 = pd.read_csv("../data/fromAPI/StaDa.csv")

In [137]:
df4.head()

,Unnamed: 0,eva_nr,name,category,state,city,zipcode,street,long,lat
0,0,8000001,Aachen Hbf,2,Nordrhein-Westfalen,Aachen,52064,Bahnhofstr. 2a,6.091499,50.767800
1,1,8000406,Aachen-Rothe Erde,4,Nordrhein-Westfalen,Aachen,52066,Beverstr. 48,6.116475,50.770202
2,2,8000404,Aachen West,5,Nordrhein-Westfalen,Aachen,52072,Republikplatz 1,6.070715,50.780360
3,3,8000002,Aalen Hbf,3,Baden-Württemberg,Aalen,73430,Am Bahnhof 1,10.096271,48.841013
4,4,8000410,Abensberg,6,Bayern,Abensberg,93326,Bahnhofstr. 27,11.846620,48.819456


In [138]:
df3["name"] = df4.query("eva_nr == 8000225").name.values[0]
df3["state"] = df4.query("eva_nr == 8000225").state.values[0]
df3["city"] = df4.query("eva_nr == 8000225").city.values[0]
df3["zipcode"] = df4.query("eva_nr == 8000225").zipcode.values[0]
df3["long"] = df4.query("eva_nr == 8000225").long.values[0]
df3["lat"] = df4.query("eva_nr == 8000225").lat.values[0]

In [139]:
df3["depature_delay_m"] = df3["depature_delay_m"].dt.total_seconds()/60
df3["depature_delay_m"] = df3["depature_delay_m"].astype(int)

In [140]:
df3["arrival_delay_m"] = df3["arrival_delay_m"].dt.total_seconds()/60
df3["arrival_delay_m"] = df3["arrival_delay_m"].astype(int)

In [142]:
df3.head()

,ID,arrival_plan,departure_plan,train,EVANr,arrival_change,departure_change,depature_delay_m,arrival_delay_m,name,state,city,zipcode,long,lat
0,8863466736973729393-2406190753-8,2024-06-19 09:14:00,2024-06-19 09:22:00,56,8000225,2024-06-19 09:15:00,2024-06-19 09:22:00,0,1,Leer (Ostfriesl),Niedersachsen,Leer,26789,7.465283,53.23165
1,-1346881581325835580-2406190500-15,2024-06-19 09:14:00,2024-06-19 09:22:00,None,8000225,2024-06-19 09:15:00,2024-06-19 09:24:00,2,1,Leer (Ostfriesl),Niedersachsen,Leer,26789,7.465283,53.23165
2,-300313760802474975-2406190852-2,2024-06-19 09:08:00,2024-06-19 09:09:00,RE15,8000225,2024-06-19 09:08:00,2024-06-19 09:09:00,0,0,Leer (Ostfriesl),Niedersachsen,Leer,26789,7.465283,53.23165
3,8982563108078925395-2406190839-6,2024-06-19 09:34:00,2024-06-19 09:41:00,1,8000225,2024-06-19 09:35:00,2024-06-19 09:42:00,1,1,Leer (Ostfriesl),Niedersachsen,Leer,26789,7.465283,53.23165
4,7350926981163673022-2406190805-15,2024-06-19 09:52:00,2024-06-19 09:53:00,RE15,8000225,2024-06-19 10:02:00,2024-06-19 10:03:00,10,10,Leer (Ostfriesl),Niedersachsen,Leer,26789,7.465283,53.23165
